In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

### Existing Data in Table

In [0]:
data = [("Sn1","Roahan",23,"Developer"),
        ("Sn2","Virat",34,"HR"),
        ("Sn3","Rohit",45,"IT Admin")]

mySchema = StructType([StructField("emp_id",StringType(),True),
                      StructField("name",StringType(),True),
                      StructField("age",IntegerType(),True),
                      StructField("designation",StringType(),True)])

target_df = spark.createDataFrame(data,mySchema)

target_df.show()

### New Coming Data along  with there operrations

In [0]:
coming_data = [("Sn1","Roahan",23,"Developer","U"),
        ("Sn4","Shyaam",39,"DBA","I"),
        ("Sn3","Rohit",45,"IT Admin","D")]

mySchema2 = StructType([StructField("emp_id",StringType(),True),
                      StructField("name",StringType(),True),
                      StructField("age",IntegerType(),True),
                      StructField("designation",StringType(),True),
                      StructField("operation",StringType(),True)])

incremental_df = spark.createDataFrame(coming_data,mySchema2)

incremental_df.show()

#### Step 1 : Filter Out All the Changed Data

In [0]:
inserted_df = incremental_df.filter(col("operation") == "I")

updated_df = incremental_df.filter(col("operation") == "U")

deleted_df = incremental_df.filter(col("operation") == "D")

inserted_df.show()
updated_df.show()
deleted_df.show()


### Handling Deleted Records
- Left Anti Join Will Remove the Deled data from table After LEFT ANTI JOIN

In [0]:
target_df.show()
incremental_df.show()

In [0]:
after_deleted_df = target_df.join(deleted_df,target_df["emp_id"] == incremental_df["emp_id"],"left_anti")

after_deleted_df.show()

### Handling The Updated Record
1. Delete The Old Data (ONLY which need to be updated) From Target Table using LEFT ANTI JOIN
2. Add Data From Updated_df after Dropping "Operation" Column

In [0]:
remove_old_data_df = after_deleted_df.join(updated_df, on = "emp_id", how = "left_anti")

remove_old_data_df.show()

after_updated_df = remove_old_data_df.union(updated_df.drop("operation"))

after_updated_df.show()

### Inserting New Data
- UNION table After_Updated_df and inserted_df after Removing "Operation" Column

In [0]:
final_output_df = after_updated_df.union(inserted_df.drop("operation"))

final_output_df.show()